In [2]:
import requests
import pandas as pd
import json

# Replace with your Scopus API key
API_KEY = "f5624674690906ad58405769d1c9b433"
BASE_URL = "https://api.elsevier.com/content/search/scopus"
headers = {
    "X-ELS-APIKey": API_KEY,
    "Accept": "application/json"
}

# Set up parameters for a single request
params = {
    "query": "all(gene)",  # Query
    "count": 25,            # Number of results to retrieve
    "start": 0,            # Start index
    "view": "STANDARD"     # View type
}


In [44]:
response = requests.get(BASE_URL, headers=headers, params=params)

In [ ]:

def get_journal_metadata(issn, api_key):
    url = f"https://api.elsevier.com/content/serial/title/issn/{issn}"
    headers = {
        "X-ELS-APIKey": api_key,
        "Accept": "application/json"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()

        if "serial-metadata-response" in data:
            entries = data["serial-metadata-response"].get("entry", [])
            if entries:
                journal_info = entries[0]
                subject_areas = journal_info.get("subject-area", [])
                if subject_areas:
                    return(data)

                else:
                    print("Subject Areas: None found.")
            else:
                print("No journal metadata found.")
        else:
            print("Invalid response format.")
    
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")






In [ ]:
all_data = []

# Check response status
if response.status_code == 200:
    data = response.json()

    # Extract entries from the response
    entries = data.get("search-results", {}).get("entry", [])

    for entry in entries:
        issn = entry.get("prism:issn", "N/A")
        journal_data = get_journal_metadata(issn, API_KEY)

        # Extract subject areas if available in the response from get_journal_metadata
        subject_area_list = []
        if journal_data and isinstance(journal_data, dict):
            metadata_response = journal_data.get("serial-metadata-response", {})
            metadata_entries = metadata_response.get("entry", [])
            
            if metadata_entries:
                subject_areas = metadata_entries[0].get("subject-area", [])
                subject_area_list = [
                    area.get("$", "Unknown") for area in subject_areas
                ]

        all_data.append({
            "date_delivered_year": entry.get("prism:coverDate", "Unknown")[:4],
            "date_sort_year": entry.get("prism:coverDate", "Unknown")[:4],
            "author_group": entry.get("author_group", "Unknown"),
            "citation_title": entry.get("dc:title", "Unknown"),
            "affiliation_country": entry.get("affiliation", [{}])[0].get("affiliation-country", "Unknown"),
            "affiliation_city": entry.get("affiliation", [{}])[0].get("affiliation-city", "Unknown"),
            "affiliation_organization": entry.get("affiliation", [{}])[0].get("affilname", "Unknown"),
            "corresponding_author_given_name": entry.get("corresponding_author_given_name", "Unknown"),
            "corresponding_author_surname": entry.get("corresponding_author_surname", "Unknown"),
            "corresponding_author_indexed_name": entry.get("dc:creator", "Unknown"),
            "citation_language": entry.get("citation_language", "Unknown"),
            "source_country": entry.get("affiliation", [{}])[0].get("affiliation-country", "Unknown"),
            "source_publication_year": entry.get("prism:coverDate", "Unknown")[:4],
            "source_publisher_name": entry.get("source_publisher_name", "Unknown"),
            "classificationgroup": entry.get("classificationgroup", "Unknown"),
            "dbcollection": entry.get("dbcollection", "Unknown"),
            "ref_count": entry.get("ref_count", "Unknown"),
            "reference": entry.get("reference", "Unknown"),
            "affiliation": entry.get("affiliation", "Unknown"),
            "coverDate": entry.get("prism:coverDate", "Unknown"),
            "aggregationType": entry.get("prism:aggregationType", "Unknown"),
            "author": entry.get("dc:creator", "Unknown"),
            "publicationName": entry.get("prism:publicationName", "Unknown"),
            "citedby_count": entry.get("citedby-count", "Unknown"),
            "title": entry.get("dc:title", "Unknown"),
            "publisher": entry.get("prism:publicationName", "Unknown"),
            "subject_area": ", ".join(subject_area_list) if subject_area_list else "Unknown",
            "abstract_language": entry.get("abstract_language", "Unknown"),
        })

else:
    print(f"Error: {response.status_code} - {response.text}")

df = pd.DataFrame(all_data)

json {
    "serial-metadata-response": {
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/serial/title/issn/03014851",
                "@type": "application/json"
            }
        ],
        "entry": [
            {
                "@_fa": "true",
                "dc:title": "Molecular Biology Reports",
                "dc:publisher": "Springer Science and Business Media B.V.",
                "coverageStartYear": "1973",
                "coverageEndYear": "1988",
                "prism:aggregationType": "journal",
                "source-id": "14154",
                "prism:issn": "0301-4851",
                "prism:eIssn": "1573-4978",
                "openaccess": null,
                "openaccessArticle": null,
                "openArchiveArticle": null,
                "openaccessType": null,
                "openaccessStartDate": null,
                "oaAllowsAuthorPaid

In [56]:
pd.set_option('display.max_columns', None)
df.head()

,date_delivered_year,date_sort_year,author_group,citation_title,affiliation_country,affiliation_city,affiliation_organization,corresponding_author_given_name,corresponding_author_surname,corresponding_author_indexed_name,citation_language,source_country,source_publication_year,source_publisher_name,classificationgroup,dbcollection,ref_count,reference,affiliation,coverDate,aggregationType,author,publicationName,citedby_count,title,publisher,subject_area,abstract_language
0,2025,2025,Unknown,Mesenchymal stem cells and mesenchymal stem ce...,Turkey,Famagusta,Eastern Mediterranean University (EMU) Famagusta,Unknown,Unknown,Hassanpour Khodaei S.,Unknown,Turkey,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,"[{'@_fa': 'true', 'affilname': 'Eastern Medite...",2025-12-01,Journal,Hassanpour Khodaei S.,Unknown,0,Mesenchymal stem cells and mesenchymal stem ce...,Molecular Biology Reports,"Molecular Biology, Genetics",Unknown


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 28 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   date_delivered_year                1 non-null      object
 1   date_sort_year                     1 non-null      object
 2   author_group                       1 non-null      object
 3   citation_title                     1 non-null      object
 4   affiliation_country                1 non-null      object
 5   affiliation_city                   1 non-null      object
 6   affiliation_organization           1 non-null      object
 7   corresponding_author_given_name    1 non-null      object
 8   corresponding_author_surname       1 non-null      object
 9   corresponding_author_indexed_name  1 non-null      object
 10  citation_language                  1 non-null      object
 11  source_country                     1 non-null      object
 12  source_publi

In [ ]:
df.to_csv("scopus_data.csv", index=False)

In [61]:

# Example Usage
issn = "10928464"  # Replace with your ISSN
api_key = "f5624674690906ad58405769d1c9b433"  # Replace with your Scopus API key
test = get_journal_metadata(issn, api_key)


json {
    "serial-metadata-response": {
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/serial/title/issn/10928464",
                "@type": "application/json"
            }
        ],
        "entry": [
            {
                "@_fa": "true",
                "dc:title": "Current Treatment Options in Cardiovascular Medicine",
                "dc:publisher": "Current Science, Inc.",
                "coverageStartYear": "1999",
                "coverageEndYear": "1999",
                "prism:aggregationType": "journal",
                "source-id": "23154",
                "prism:issn": "1092-8464",
                "prism:eIssn": "1534-3189",
                "openaccess": null,
                "openaccessArticle": null,
                "openArchiveArticle": null,
                "openaccessType": null,
                "openaccessStartDate": null,
                "oaAllowsAu

In [63]:
import requests
import pandas as pd

# Replace with your Scopus API key
API_KEY = "f5624674690906ad58405769d1c9b433"
BASE_URL = "https://api.elsevier.com/content/search/scopus"
headers = {
    "X-ELS-APIKey": API_KEY,
    "Accept": "application/json"
}

# Function to get journal metadata
def get_journal_metadata(issn, api_key):
    url = f"https://api.elsevier.com/content/serial/title/issn/{issn}"
    headers = {
        "X-ELS-APIKey": api_key,
        "Accept": "application/json"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()

        if "serial-metadata-response" in data:
            entries = data["serial-metadata-response"].get("entry", [])
            if entries:
                return data
            else:
                print(f"No journal metadata found for ISSN: {issn}")
        else:
            print("Invalid response format.")
    
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")

# Set up parameters for the request
params = {
    "query": "all(gene)",  # Query
    "count": 25,            # Number of results to retrieve per request
    "start": 0,             # Start index
    "view": "STANDARD"      # View type
}

all_data = []

# Loop to extract 1000 records (40 requests of 25 results each)
for start_index in range(0, 25, 25):
    params["start"] = start_index
    response = requests.get(BASE_URL, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()

        # Extract entries from the response
        entries = data.get("search-results", {}).get("entry", [])

        for entry in entries:
            issn = entry.get("prism:issn", "N/A")
            journal_data = get_journal_metadata(issn, API_KEY)

            # Extract subject areas if available in the response from get_journal_metadata
            subject_area_list = []
            if journal_data and isinstance(journal_data, dict):
                metadata_response = journal_data.get("serial-metadata-response", {})
                metadata_entries = metadata_response.get("entry", [])
                
                if metadata_entries:
                    subject_areas = metadata_entries[0].get("subject-area", [])
                    subject_area_list = [
                        area.get("$", "Unknown") for area in subject_areas
                    ]

            # Collect data from the entry and append to the list
            all_data.append({
                "date_delivered_year": entry.get("prism:coverDate", "Unknown")[:4],
                "date_sort_year": entry.get("prism:coverDate", "Unknown")[:4],
                "author_group": entry.get("author_group", "Unknown"),
                "citation_title": entry.get("dc:title", "Unknown"),
                "affiliation_country": entry.get("affiliation", [{}])[0].get("affiliation-country", "Unknown"),
                "affiliation_city": entry.get("affiliation", [{}])[0].get("affiliation-city", "Unknown"),
                "affiliation_organization": entry.get("affiliation", [{}])[0].get("affilname", "Unknown"),
                "corresponding_author_given_name": entry.get("corresponding_author_given_name", "Unknown"),
                "corresponding_author_surname": entry.get("corresponding_author_surname", "Unknown"),
                "corresponding_author_indexed_name": entry.get("dc:creator", "Unknown"),
                "citation_language": entry.get("citation_language", "Unknown"),
                "source_country": entry.get("affiliation", [{}])[0].get("affiliation-country", "Unknown"),
                "source_publication_year": entry.get("prism:coverDate", "Unknown")[:4],
                "source_publisher_name": entry.get("source_publisher_name", "Unknown"),
                "classificationgroup": entry.get("classificationgroup", "Unknown"),
                "dbcollection": entry.get("dbcollection", "Unknown"),
                "ref_count": entry.get("ref_count", "Unknown"),
                "reference": entry.get("reference", "Unknown"),
                "affiliation": entry.get("affiliation", "Unknown"),
                "coverDate": entry.get("prism:coverDate", "Unknown"),
                "aggregationType": entry.get("prism:aggregationType", "Unknown"),
                "author": entry.get("dc:creator", "Unknown"),
                "publicationName": entry.get("prism:publicationName", "Unknown"),
                "citedby_count": entry.get("citedby-count", "Unknown"),
                "title": entry.get("dc:title", "Unknown"),
                "publisher": entry.get("prism:publicationName", "Unknown"),
                "subject_area": ", ".join(subject_area_list) if subject_area_list else "Unknown",
                "abstract_language": entry.get("abstract_language", "Unknown"),
            })

    else:
        print(f"Error: {response.status_code} - {response.text}")

# Create DataFrame and save as CSV
df = pd.DataFrame(all_data)
df.to_csv("scopus_data_1000.csv", index=False)

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
file_path = "scopus_data_1000.csv"
pd.set_option('display.max_columns', None)
df = pd.read_csv(file_path)

# Save the updated DataFrame back to the CSV file
df.to_csv(file_path, index=False)
print(df.head(20))
df.info()

    date_delivered_year  date_sort_year author_group  \
0                  2024            2024      Unknown   
1                  2024            2024      Unknown   
2                  2024            2024      Unknown   
3                  2024            2024      Unknown   
4                  2024            2024      Unknown   
5                  2024            2024      Unknown   
6                  2024            2024      Unknown   
7                  2024            2024      Unknown   
8                  2024            2024      Unknown   
9                  2024            2024      Unknown   
10                 2024            2024      Unknown   
11                 2024            2024      Unknown   
12                 2024            2024      Unknown   
13                 2024            2024      Unknown   
14                 2024            2024      Unknown   
15                 2024            2024      Unknown   
16                 2024            2024      Unk

In [9]:

# Read the CSV file into a DataFrame
file_path = "scopus_data_1000.csv"
pd.set_option('display.max_columns', None)
df = pd.read_csv(file_path)
filtered_df = df[df['date_delivered_year'] == 2024]
print(filtered_df.head())

   date_delivered_year  date_sort_year author_group  \
0                 2024            2024      Unknown   
1                 2024            2024      Unknown   
2                 2024            2024      Unknown   
3                 2024            2024      Unknown   
4                 2024            2024      Unknown   

                                      citation_title affiliation_country  \
0  Mesenchymal stem cells and mesenchymal stem ce...              Turkey   
1  The role of the mucosal barrier system in main...               Japan   
2  Fabrication of curcumin-incorporated human amn...                Iran   
3  Analysis of Alzheimer’s disease associated del...              Turkey   
4  A Fully-Printed Wearable Bandage-Based Electro...            Thailand   

  affiliation_city                          affiliation_organization  \
0        Famagusta  Eastern Mediterranean University (EMU) Famagusta   
1            Suita                       Graduate School of Medicine